In [1]:
import napari
import pandas as pd
import numpy as np
import glob
import skimage as ski
import scipy.ndimage as ndi
import plotly.express as px
import cv2
from roifile import ImagejRoi

In [2]:
viewer = napari.Viewer()

In [3]:
folder_of_rois = 'Cen7_ROI/*.zip'
channel_to_quantify = 1

# Loading files

Will use the ImageJ roi files, as they are easy to open with the ImagejRoi python module

In [4]:
pos_files = glob.glob(folder_of_rois)
pos_files.sort()
pos_files

['Cen7_ROI\\Image001.zip',
 'Cen7_ROI\\Image002.zip',
 'Cen7_ROI\\Image003a.zip',
 'Cen7_ROI\\Image003b.zip',
 'Cen7_ROI\\Image004.zip',
 'Cen7_ROI\\Image005.zip',
 'Cen7_ROI\\Image006a.zip',
 'Cen7_ROI\\Image006b.zip',
 'Cen7_ROI\\Image007a.zip',
 'Cen7_ROI\\Image007b.zip',
 'Cen7_ROI\\Image007c.zip',
 'Cen7_ROI\\Image008.zip',
 'Cen7_ROI\\Image009a.zip',
 'Cen7_ROI\\Image009b.zip',
 'Cen7_ROI\\Image010a.zip',
 'Cen7_ROI\\Image010b.zip',
 'Cen7_ROI\\Image011.zip',
 'Cen7_ROI\\Image012a.zip',
 'Cen7_ROI\\Image012b.zip',
 'Cen7_ROI\\Image013.zip',
 'Cen7_ROI\\Image014.zip',
 'Cen7_ROI\\Image015a.zip',
 'Cen7_ROI\\Image015b.zip',
 'Cen7_ROI\\Image016a.zip',
 'Cen7_ROI\\Image016b.zip',
 'Cen7_ROI\\Image017a.zip',
 'Cen7_ROI\\Image017b.zip',
 'Cen7_ROI\\Image018a.zip',
 'Cen7_ROI\\Image018b.zip',
 'Cen7_ROI\\Image019.zip',
 'Cen7_ROI\\Image020a.zip',
 'Cen7_ROI\\Image020b.zip']

### Test example

In [5]:
fname = pos_files[15]

In [6]:
im_name = fname.replace('Cen7_ROI\\', 'SUM_')[0:12]+'.tif'
img = ski.io.imread(im_name)

In [7]:
viewer.add_image(img, channel_axis=2)

[<Image layer 'Image' at 0x233a6357070>,
 <Image layer 'Image [1]' at 0x233a6356770>,
 <Image layer 'Image [2]' at 0x233a8365a20>]

# Analysis functions

For doing background subtraction on the images we mean to quantify.

In [8]:
def backsub(inp, radius=10):
    filterSize =(radius, radius)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,
                                    filterSize)
    blurred = cv2.GaussianBlur(inp, (5, 5), 0)
    tophat_img = cv2.morphologyEx(blurred,
                                cv2.MORPH_TOPHAT,
                                kernel)
    rtn = inp.astype(np.single) - (blurred-tophat_img)
    rtn = np.clip(rtn, 0, np.inf)

    return rtn

This takes an roi.zip file as an input, loads the associated image, and creates a new image (paint_img) which is a label image where the labels are lines (indices 1 and 2) connecting the two points in the roi file.

In [9]:
def load_images(fname, display=False):
    im_name = fname.replace('Cen7_ROI\\', 'SUM_')[0:12]+'.tif'
    img = ski.io.imread(im_name)

    paint_img = np.zeros_like(img[:,:,channel_to_quantify])
    roi_list = ImagejRoi.fromfile(fname)
    for idx, roi in enumerate(roi_list):
        coords = roi.coordinates()
        cv2.line(paint_img, tuple(coords[0].astype(int)), tuple(coords[1].astype(int)), (idx+1), 3)

    if display:
        viewer.layers.clear()
        viewer.add_image(img[:,:,channel_to_quantify], name='Original')
        viewer.add_labels(paint_img.astype(int), name='Lines')
    return img, paint_img

This takes the raw image channel to be quantified, as well as the label image with the lines drawn on it.  It performs background subtraction on the raw, then smooths and finds a threshold using the otsu algorithm (same as ImageJ).  I could not use a constant value for threshold, as some images were much much brighter than others, and doing so led to huge discrepancies in the regions quantified.  I also found the un-adjusted otsu was a bit too selective, so I dilute it by a factor of 3/4 by default.  

I was finding that signal from different lines were frequently getting merged together as the lines were often adjacent or touching one another.  To get around this I use traditional watershedding where the labeled image of the lines is used as a seed.  This works reasonably well at separating the regions even if they get segmented together into a single blob.

The function ireturns two images:  the background subtracted image (before smoothing) to be used for quantification and the label image of each region of interest.  

In [10]:
def prepare_image(raw_img, paint_img, threshold=1.0, display=False):
    backsub_img = backsub(raw_img, 10)
    smoothed_img = ndi.gaussian_filter(backsub_img, 1)
    otsu_thresh = ski.filters.threshold_otsu(smoothed_img)
    otsu_thresh = otsu_thresh*threshold
    thresholded = smoothed_img > otsu_thresh

    # This is the watershedding step, first we compute the distance image from our thresholded image, then
    # use that to decide whether a pixel is closer to one line or another
    edt = ndi.distance_transform_edt(thresholded)
    watershedded = ski.segmentation.watershed(-edt, paint_img.astype(int), mask=thresholded)
    
    if display:
        viewer.add_image(backsub_img, name='Backsub', blending='additive')
        viewer.add_image(smoothed_img, name='Smoothed', blending='additive')
        viewer.add_labels(thresholded, name='Thresholded', blending='additive')
        viewer.add_labels(watershedded, name='Watershed', blending='additive')
    return backsub_img, watershedded


### Testing whole pipeline

This will check the roi in pos_files[0] and display the results in napari, it will quantify the intensity image using the labels from prepare_image

In [11]:
img, paint_img = load_images(pos_files[15], display=True)
quant_img, labels = prepare_image(img[:,:,channel_to_quantify], paint_img, display=True) 

df = pd.DataFrame(ski.measure.regionprops_table(labels, intensity_image=quant_img, properties=('label', 'area', 'mean_intensity')))

In [12]:
df

,label,area,mean_intensity
0,1,47.0,4510.882324
1,2,75.0,4886.607422


### Run on all rois

In [13]:
master_df = pd.DataFrame()
for fname in pos_files:
    img, paint_img = load_images(fname)
    quant_img, labels = prepare_image(img[:,:,channel_to_quantify], paint_img) 
    df = pd.DataFrame(ski.measure.regionprops_table(labels, intensity_image=quant_img, properties=('label', 'area', 'mean_intensity')))

    # Add a column for the integrated density, and another for the file name
    df['IntDensity'] = df['area'] * df['mean_intensity']
    df['file'] = fname
    master_df = pd.concat([master_df, df])
df = master_df

The file name has lots of useless junk in it, we want just which image it came from.

In [14]:
df['Image'] = df['file'].str[9:17]

# Plotting results

In [15]:
import plotly.graph_objs as go
import plotly.express as px

f=go.FigureWidget(
    px.box(df, x='Image', y='IntDensity', color='label', points='all', hover_data=['file'])
    )

def click_fn(trace, points, state):
    
    if (len(points.point_inds)>0):
        idx = f.data[points.trace_index]['customdata'][points.point_inds[-1]][0]
        print(idx)

        # If we click on a spot, we want to display the image and the lines
        img, paint_img = load_images(idx, display=True)
        quant_img, labels = prepare_image(img[:,:,channel_to_quantify], paint_img, display=True)

        # I turn off most of the layers by default as they are diagnostic
        for layer in viewer.layers:
            layer.visible = False
        viewer.layers[0].visible = True
        viewer.layers[-1].visible = True
        viewer.layers[-1].contour = 2

for a in f.data:
    a.on_click(click_fn)

f

FigureWidget({
    'data': [{'alignmentgroup': 'True',
              'boxpoints': 'all',
              'customdata': array([['Cen7_ROI\\Image001.zip'],
                                   ['Cen7_ROI\\Image002.zip'],
                                   ['Cen7_ROI\\Image003a.zip'],
                                   ['Cen7_ROI\\Image003b.zip'],
                                   ['Cen7_ROI\\Image004.zip'],
                                   ['Cen7_ROI\\Image005.zip'],
                                   ['Cen7_ROI\\Image006a.zip'],
                                   ['Cen7_ROI\\Image006b.zip'],
                                   ['Cen7_ROI\\Image007a.zip'],
                                   ['Cen7_ROI\\Image007b.zip'],
                                   ['Cen7_ROI\\Image007c.zip'],
                                   ['Cen7_ROI\\Image008.zip'],
                                   ['Cen7_ROI\\Image009a.zip'],
                                   ['Cen7_ROI\\Image009b.zip'],
                    

# Outputting results

Save quantifications to csv file

In [16]:
df.to_csv('Results_BackSub10' + str(channel_to_quantify) + '.csv')

Save graph as html file

In [17]:
f = px.box(df, x='Image', y='IntDensity', color='label', points='all', hover_data=['file'])
f.write_html('boxplot_BackSub60' + str(channel_to_quantify) + '.html')